In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from src.folder_handler import *
from src.cort_processor import *
from src.cca_processor import *
from src.tdt_support import *
from src.plotter import *
from src.decoders import *
from src.utils import *
import pickle
import scipy as sio
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
from src.wiener_filter import *
from matplotlib.pyplot import cm
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, LinearRegression
import os
from itertools import cycle, islice
import copy

In [2]:
path = '/home/diya/Documents/rat-fes/data/filipe_data/N5'
filenames = os.listdir(path)
print(filenames)

['N5_171130_No Obstacles_s.mat', 'N5_171016_No Obstacles_s.mat', 'N5_171001_No Obstacles_s.mat', 'N5_170929_No Obstacles_s.mat']


In [3]:
temp_datasets = []
temp_var_names = []

for file in filenames:
    temp_datasets.append(path+'/'+file)
    temp_var_names.append(file.split('_')[0]+file.split('_')[1])

datasets = sorted(temp_datasets)
var_names = sorted(temp_var_names)

In [4]:
cort_dict = {}
for i in range(len(datasets)):
    cort_dict[var_names[i]] = CortProcessor(datasets[i])

this is filipe data i belive
this is filipe data i belive
this is filipe data i belive
this is filipe data i belive


/home/diya/anaconda3/envs/rat-fes/lib/python3.7/site-packages/scipy/io/matlab/mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [5]:
cca_dict = {}
for i in range(1, len(var_names)):
    print(var_names[0], var_names[i])
    cca_dict[var_names[i]] = CCAProcessor(cort_dict[var_names[0]], cort_dict[var_names[i]], align=0)


N5170929 N5171001
(12050,)
(12039,)
should be good to align
(10267, 11)
(10267, 5)
(10267, 23)
(10267, 5)
N5170929 N5171016
(12050,)
(12127,)
should be good to align
(8680, 11)
(8680, 5)
(8680, 22)
(8680, 5)
N5170929 N5171130
(12050,)
(12051,)
should be good to align
(9543, 11)
(9543, 5)
(9543, 23)
(9543, 5)


In [6]:
day0_decoder, vaffy, test_x, test_y = cort_dict[var_names[0]].decode_angles()

In [7]:
dayn_x_list = []
for i in range(1, len(var_names)):
    dayn_x_list.append(cca_dict[var_names[i]].remove_cp2_channels())
    

fixed_vaf_list = []
same_day_vaf = []
for i in range(0, len(var_names)):
    temp_cp = cort_dict[var_names[i]]
    x = dayn_x_list[i-1]
    y = temp_cp.data['angles'][0]
    
    x_format, y_format = format_data(x, y)
    nada, temp_vaf, nadax, naday = decode_kfolds(x_format, y_format, metric=1, preset_h =day0_decoder)
    fixed_vaf_list.append(np.average(temp_vaf,1)[1])
    nada, temp_vaf, nadax, naday = temp_cp.decode_angles(X=temp_cp.apply_PCA(dims=8))
    same_day_vaf.append(np.average(temp_vaf,1)[1])
    
fixed_vaf_list[0] = .563

In [8]:
fixed_vaf_list

[0.563, 0.136, 0.11699999999999999, 0.429]

In [9]:
temp_x = cort_dict[var_names[0]].apply_PCA(dims=8)
day0_pca_decoder, nada, nadas, nasd = cort_dict[var_names[0]].decode_angles(X=temp_x)

In [10]:
temp_x[0].shape

(12050, 8)

In [11]:
cca_vaf = []
for i in range(1, 2):
    pca_x1, pca_x2 = cca_dict[var_names[i]].apply_PCA(preset_num_components=8)
    temp_cca_transformer, nada = cca_dict[var_names[i]].apply_CCA(cp1_x = pca_x1, cp2_x = pca_x2, preset_num_components = 8, pca=True)
    temp_test_x = cca_dict[var_names[i]].cp2.data['rates']
    test_x = cca_dict[var_names[i]].cp2.apply_PCA(dims=8, X=temp_test_x)
    test_y = cca_dict[var_names[i]].cp2.data['angles'][0]
    trans_x = cca_dict[var_names[i]].quick_cca(x=test_x[0], transformer = temp_cca_transformer)
    
    trans_x_format, test_y_format = format_data(trans_x, test_y)
    predic = test_wiener_filter(trans_x_format, day0_pca_decoder)
    temp_vaf = vaf(test_y_format[:,1], predic[:,1])
    cca_vaf.append(temp_vaf)
    

dim0 corr is 0.45
dim1 corr is 0.17
dim2 corr is 0.14


In [12]:
cca_decoder_scores = []
cca_transformers=[]
for i in range(1, len(var_names)):
    #make subsampled cp2
    pca_proc_x1, pca_proc_x2 = cca_dict[var_names[i]].apply_PCA(preset_num_components=8)
    temp_cca_transformer, nada = cca_dict[var_names[i]].apply_CCA(preset_num_components = 8, pca=True)
    
    cca_transformers.append(temp_cca_transformer)
    
    test_x = np.squeeze(np.array(cort_dict[var_names[i]].apply_PCA(dims=8, transformer = cca_dict[var_names[i]].data['cp2']['pca_transformer'])))
    test_y = np.squeeze(np.array(cort_dict[var_names[i]].data['angles']))
    
    nada, test_x_cca_space = temp_cca_transformer.transform(test_x, test_x)
    
    temp_x = temp_cca_transformer.inverse_transform(test_x_cca_space)
    
    temp_x_format, test_y_format = format_data(temp_x, test_y)
    predic = test_wiener_filter(temp_x_format, day0_pca_decoder)
    
    cca_decoder_scores.append(vaf(test_y_format[:,1], predic[:,1]))

dim0 corr is 0.45
dim1 corr is 0.17
dim2 corr is 0.14
dim0 corr is 0.46
dim1 corr is 0.15
dim2 corr is 0.11
dim0 corr is 0.49
dim1 corr is 0.18
dim2 corr is 0.14


In [13]:
cca_decoder_scores

[0.64, 0.42, 0.6]

In [14]:
fixed_vaf_list

[0.563, 0.136, 0.11699999999999999, 0.429]

In [15]:
same_day_vaf

[0.563, 0.773, 0.6849999999999999, 0.8039999999999999]

In [ ]:
fig, ax = plt.subplots()
ax.bar(